In [1]:
"""
Description: Script to optimize Duca's currency mix.
Author: Jeroen van Dijk & Victor de Graaff
Date: 04-11-2020
Maintainer: Jeroen van Dijk & Victor de Graaff
Email: jeroen.vandijk@d-data.nl & victor.degraaff@d-data.nl
Status: Dev
"""

"\nDescription: Script to optimize Duca's currency mix.\nAuthor: Jeroen van Dijk & Victor de Graaff\nDate: 04-11-2020\nMaintainer: Jeroen van Dijk & Victor de Graaff\nEmail: jeroen.vandijk@d-data.nl & victor.degraaff@d-data.nl\nStatus: Dev\n"

In [2]:
# Load common imports
%run ./CommonImports.ipynb

In [3]:
# Load common functions and currencies lists:
# - all_currencies
# - obsolete_currencies
# - p13_currencies
# - f_currencies
# - ff_currencies
# - currencies_per_continent
%run ./Utilities.ipynb

In [4]:
# Load loss functions
# - calculate_loss_function_around_one(weights)
# - calculate_loss_function_vs_t_minus_one(weights)
# - calculate_loss_function_vs_t_minus_one_with_balancing(weights)
# - calculate_loss_function_vs_t_minus_one_for_period(weights, max_date)
%run ./LossFunctions.ipynb

In [5]:
# Create dataframe with pre-defined base currency
df = create_original_df("EUR")

In [6]:
# Determine splits for dev- (train and test) and validation set
train_start = date(2005, 4, 1)
test_start = date(2012, 1, 1)
validation_start = date(2015, 1, 1)
validation_end = date(2020, 10, 1)

# Create dev (train and test) and validation set
train, test, dev, val = split_data(df, train_start, test_start, validation_start, validation_end)

In [7]:
# Create exchange table for each currency
exchange_table = dict()
for base_currency in all_currencies:
    exchange_table[base_currency] = 1/train[all_currencies].divide(train[base_currency], axis=0)

# Preview USD exchange table
exchange_table["USD"]

,AUD,EUR,NOK,HRK,NZD,PHP,ZAR,GBP,MYR,RUB,TRY,SGD,RON,CAD,USD,CNY,CHF,HUF,PLN,IDR,HKD,THB,CZK,SEK,JPY,DKK,KRW
date,,,,,,,,,,,,,,,,,,,,,,,,,,,
2005-04-01,0.77123,1.29590,0.15813,0.17460,0.71051,0.01827,0.16175,1.88728,0.26316,0.03588,0.73963,0.60418,0.35967,0.82353,1.00000,0.12082,0.83461,0.00524,0.31703,0.00011,0.12823,0.02554,0.04315,0.14134,0.00932,0.17394,0.00099
2005-04-02,0.77012,1.29337,0.15780,0.17424,0.70945,0.01826,0.16153,1.88327,0.26316,0.03587,0.73758,0.60293,0.35897,0.82270,1.00000,0.12082,0.83284,0.00523,0.31580,0.00011,0.12822,0.02546,0.04307,0.14100,0.00930,0.17360,0.00099
2005-04-03,0.76901,1.29083,0.15748,0.17387,0.70838,0.01825,0.16132,1.87926,0.26317,0.03586,0.73553,0.60168,0.35827,0.82187,1.00000,0.12082,0.83106,0.00522,0.31458,0.00011,0.12822,0.02537,0.04299,0.14065,0.00928,0.17326,0.00099
2005-04-04,0.76790,1.28830,0.15715,0.17351,0.70731,0.01824,0.16110,1.87525,0.26317,0.03585,0.73349,0.60044,0.35756,0.82104,1.00000,0.12082,0.82929,0.00521,0.31336,0.00011,0.12821,0.02529,0.04291,0.14031,0.00926,0.17292,0.00099
2005-04-05,0.76441,1.28100,0.15691,0.17266,0.70431,0.01824,0.16014,1.87253,0.26316,0.03579,0.73137,0.60059,0.35554,0.81681,1.00000,0.12082,0.82427,0.00518,0.31019,0.00011,0.12821,0.02522,0.04271,0.13960,0.00921,0.17195,0.00098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011-12-27,1.01609,1.30690,0.16767,0.17385,0.77395,0.02285,0.12258,1.56797,0.31511,0.03197,0.52632,0.77144,0.30464,0.98101,1.00000,0.15816,1.07035,0.00428,0.29675,0.00011,0.12859,0.03181,0.05068,0.14564,0.01284,0.17582,0.00087
2011-12-28,1.01989,1.30740,0.16764,0.17367,0.77766,0.02274,0.12334,1.56725,0.31596,0.03159,0.52633,0.77197,0.30502,0.98530,1.00000,0.15827,1.07199,0.00426,0.29749,0.00011,0.12858,0.03171,0.05064,0.14578,0.01288,0.17583,0.00087
2011-12-29,1.00703,1.28890,0.16580,0.17100,0.76652,0.02281,0.12265,1.54175,0.31451,0.03103,0.52288,0.76697,0.29957,0.97725,1.00000,0.15822,1.05769,0.00415,0.29022,0.00011,0.12866,0.03147,0.04975,0.14415,0.01286,0.17337,0.00087


In [8]:
reference_date_exhange_rate = train[all_currencies].iloc[0]

In [55]:
def run_optimization(selected_currencies, 
                     max_iter=100, 
                     loss_function=calculate_loss_function_around_one):
    starting_point = np.ones(len(selected_currencies))/len(selected_currencies)
    bounds = [(0, 1) for _ in range(len(starting_point))]
    
    result = minimize(fun=loss_function, 
                      x0=starting_point, 
                      args=selected_currencies,
                      bounds=bounds,
                      options={"disp": True, 
                               "maxiter": max_iter})
    
    output = pd.Series(result.x/result.x.sum(), index=reference_date_exhange_rate.loc[selected_currencies].index)
    output = output.sort_values(ascending=False)
        
    return output

In [56]:
def try_currencies(currencies_to_try, currencies):
    for currency in currencies_to_try:
        if currency in currencies:
            return None, 1

    print(f"Evaluating {currencies} + {currencies_to_try}")
    new_mix = run_optimization(selected_currencies=currencies + currencies_to_try, 
                               max_iter=100, 
                               loss_function=calculate_loss_function_around_one)

    new_score = calculate_loss_function_around_one(new_mix, currencies + currencies_to_try)

    return new_mix, new_score

In [ ]:
best_score = 1
last_score = 1
currencies = ['USD', 'NOK', 'DKK', 'AUD', 'JPY', 'GBP']

uncoupled_currencies = [[c] for c in all_currencies if c != "HKD"]
uncoupled_currencies.sort()

currency_pairs = [[c1[0], c2[0]] for c1 in uncoupled_currencies for c2 in uncoupled_currencies if c1 > c2]
currency_pairs.sort()

while best_score == 1 or currencies_to_add is not None:
    print(f"Attempting to improve set, starting from: {currencies}")
    currencies_to_add = None
    
    for currencies_to_try in uncoupled_currencies + currency_pairs:
        new_mix, new_score = try_currencies(currencies_to_try, currencies)

        if new_score < .9995 * last_score:
            if new_score < best_score:
                best_score = new_score
                currencies_to_add = currencies_to_try

                print(f"New best score found: {new_score}")
                print(new_mix)
            elif last_score < 1:
                print(f"Better than last score, but not better than best: {new_score}")
                print(new_mix)
    
    last_score = best_score
    
    if currencies_to_add is not None:
        currencies += currencies_to_add
        
print(f"Done. Best set: {currencies}")

Attempting to improve set, starting from: ['USD', 'NOK', 'DKK', 'AUD', 'JPY', 'GBP']
Evaluating ['USD', 'NOK', 'DKK', 'AUD', 'JPY', 'GBP'] + ['CAD']
New best score found: 0.007004112945263491
USD   0.51104
DKK   0.19666
GBP   0.09393
JPY   0.09285
AUD   0.06221
CAD   0.03486
NOK   0.00845
dtype: float64
Evaluating ['USD', 'NOK', 'DKK', 'AUD', 'JPY', 'GBP'] + ['CHF']
Evaluating ['USD', 'NOK', 'DKK', 'AUD', 'JPY', 'GBP'] + ['CNY']
Evaluating ['USD', 'NOK', 'DKK', 'AUD', 'JPY', 'GBP'] + ['CZK']
Evaluating ['USD', 'NOK', 'DKK', 'AUD', 'JPY', 'GBP'] + ['EUR']
Evaluating ['USD', 'NOK', 'DKK', 'AUD', 'JPY', 'GBP'] + ['HRK']
Evaluating ['USD', 'NOK', 'DKK', 'AUD', 'JPY', 'GBP'] + ['HUF']
Evaluating ['USD', 'NOK', 'DKK', 'AUD', 'JPY', 'GBP'] + ['IDR']
Evaluating ['USD', 'NOK', 'DKK', 'AUD', 'JPY', 'GBP'] + ['KRW']
Evaluating ['USD', 'NOK', 'DKK', 'AUD', 'JPY', 'GBP'] + ['MYR']
Evaluating ['USD', 'NOK', 'DKK', 'AUD', 'JPY', 'GBP'] + ['NZD']
Evaluating ['USD', 'NOK', 'DKK', 'AUD', 'JPY', 'GBP'] +

In [ ]:
# New best score found: 0.007005720087502464
# USD   0.54671
# DKK   0.19379
# AUD   0.08139
# JPY   0.07595
# GBP   0.06701
# NOK   0.03515


# New best score found: 0.00700350782006296
# USD   0.51866
# DKK   0.17968
# JPY   0.09762
# GBP   0.09432
# AUD   0.06527
# CAD   0.02763
# PLN   0.01515
# NOK   0.00166

In [52]:
mix3 = run_optimization(selected_currencies=['NZD', 'EUR', 'HRK'], 
                                   max_iter=100, 
                                   loss_function=calculate_loss_function_around_one)

In [53]:
calculate_loss_function_around_one(mix3.values, mix3.index) / 0.007002770452103174

1.2428404205248926